In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) 

In [61]:
from PIL import Image

def weighted_blend(image1, image2, weight1, weight2):
    """
    Blend two grayscale images using weighted blending.

    Args:
    - image1: PIL image object of the first grayscale image
    - image2: PIL image object of the second grayscale image
    - weight1: Weight assigned to the first image (between 0 and 1)
    - weight2: Weight assigned to the second image (between 0 and 1)

    Returns:
    - blended_image: PIL image object of the blended image
    """
    # Convert images to grayscale mode (L mode)
    image1 = image1.convert("L")
    image2 = image2.convert("L")

    # Ensure images have the same size
    if image1.size != image2.size:
        raise ValueError("Images must have the same size")

    # Blend the images pixel by pixel
    blended_pixels = []
    for pixel1, pixel2 in zip(image1.getdata(), image2.getdata()):
        # Blend pixel values using the provided weights
        blended_pixel = int(pixel1 * weight1 + pixel2 * weight2)
        blended_pixels.append(blended_pixel)

    # Create a new image with the blended pixels
    blended_image = Image.new("L", image1.size)
    blended_image.putdata(blended_pixels)

    return blended_image

# Example usage:
if __name__ == "__main__":
    # Load grayscale images
    image1 = Image.open("C:/brain_tumor/version2/notebook/try_img/image_10.png")
    image2 = Image.open("C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_15_1.png")

    # Define weights
    weight1 = 0.3  # Weight for image1
    weight2 = 0.7  # Weight for image2

    # Perform weighted blending
    blended_image = weighted_blend(image1, image2, weight1, weight2)

    # Save or display the blended image
    blended_image.show()
    # blended_image.save("blended_image.png")


In [46]:
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter

def sobel_edge(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Apply Sobel filter
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    # Combine x and y gradients
    sobel_edge = np.sqrt(sobel_x**2 + sobel_y**2)
    # Normalize to range [0, 1]
    sobel_edge = sobel_edge / np.max(sobel_edge)
    return sobel_edge

def gaussian_weight(image, sigma=1):
    # Apply Gaussian filter
    blurred_image = gaussian_filter(image, sigma=sigma)
    return blurred_image

def combine_images(image1, image2, alpha=0.5):
    # Blend two images using weighted sum
    combined_image = cv2.addWeighted(image1, alpha, image2, 1-alpha, 0)
    return combined_image

# Example usage
if __name__ == "__main__":
    # Load two generated images from different GAN models
    image1 = cv2.imread("C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png")
    image2 = cv2.imread("C:/brain_tumor/version2/notebook/try_img/image_10.png")

    # Apply Sobel edge detection
    edge_image1 = sobel_edge(image1)
    edge_image2 = sobel_edge(image2)

    # Apply Gaussian weighting
    weighted_image1 = gaussian_weight(image1)
    weighted_image2 = gaussian_weight(image2)

    # Combine the processed images
    alpha = 0.5  # Adjust the blending ratio as needed
    final_image = combine_images(weighted_image1, weighted_image2, alpha)

    # Display or save the final synthesized image
    cv2.imshow("Final Synthesized Image", final_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [69]:
import cv2
import numpy as np

image1 = cv2.imread('C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_15_1.png')
image2 = cv2.imread('C:/brain_tumor/version2/notebook/try_img/image_10.png')

def calculate_gaussian_intensity(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to smooth the image
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    return blurred_image

def calculate_weights(image1, image2):
    # Calculate Gaussian intensity values for edge-mapped images
    gaussian_intensity1 = calculate_gaussian_intensity(image1)
    gaussian_intensity2 = calculate_gaussian_intensity(image2)
    
    # Normalize Gaussian intensity values
    normalized_intensity1 = gaussian_intensity1 / np.max(gaussian_intensity1)
    normalized_intensity2 = gaussian_intensity2 / np.max(gaussian_intensity2)
    
    # Handle division by zero
    denominator = normalized_intensity1 + normalized_intensity2
    denominator[denominator == 0] = 1  # Replace zeros with 1 to avoid division by zero
    # Calculate weights based on Gaussian intensity values
    weight1 = normalized_intensity1 / denominator
    weight2 = normalized_intensity2 / denominator
    
    # Convert weights to the appropriate data type
    weight1 = np.float32(weight1)
    weight2 = np.float32(weight2)
    
    return weight1, weight2


def aggregate_images(image1, image2, weight1, weight2):
    # Aggregate the two images pixel-wise based on the calculated weights
    aggregated_image = cv2.addWeighted(image1, 1.0, image2, 1.0, 0)
    return aggregated_image

# Calculate weights
weight1, weight2 = calculate_weights(image1, image2)
# Aggregate images
aggregated_image = aggregate_images(image1, image2, weight1, weight2)
# Display or save the aggregated image
cv2.imshow("Aggregated Image", aggregated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [68]:
import cv2
import numpy as np
from skimage.filters import sobel
from scipy.ndimage import gaussian_filter

# Function to calculate weights
def calculate_weights(g1, g2, metric1, metric2):
    denominator = g1 + g2
    w = np.where(denominator != 0, g1 / denominator, metric1 / (metric1 + metric2))
    return w

# Load your images
image1 = cv2.imread("C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png", cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread("C:/brain_tumor/version2/notebook/try_img/image_10.png", cv2.IMREAD_GRAYSCALE)

# Apply Sobel filters using scikit-image
edge_image1 = sobel(image1)
edge_image2 = sobel(image2)

# Apply Gaussian filter using SciPy
sigma = 1.0
edge_image1_gaussian = gaussian_filter(edge_image1, sigma=sigma)
edge_image2_gaussian = gaussian_filter(edge_image2, sigma=sigma)

# Calculate weights
g1_matrix = np.exp(-(edge_image1_gaussian ** 2) / (2 * (sigma ** 2)))
g2_matrix = np.exp(-(edge_image2_gaussian ** 2) / (2 * (sigma ** 2)))

metric_image1 = 0.5542362183803938
metric_image2 = 0.6147776098315835

weight_matrix_1 = calculate_weights(g1_matrix, g2_matrix, metric_image1, metric_image2)
weight_matrix_2 = calculate_weights(g2_matrix, g1_matrix, metric_image2, metric_image1)

# Aggregate images
aggregated_image = image1 * weight_matrix_1 + image2 * weight_matrix_2

# Display or save the aggregated image
cv2.imshow("Aggregated Image", aggregated_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [65]:
import cv2
import numpy as np

def image_fusion(img1, img2):
    # Convert images to grayscale
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Resize images to have the same dimensions (if needed)
    # You can use cv2.resize() or other methods for resizing
    
    # Perform fusion (simple averaging for pixel-level fusion)
    fused_image = cv2.addWeighted(gray_img1, 0.5, gray_img2, 0.5, 0)
    
    return fused_image

# Load medical images
image1_path = 'C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png'  # Update with your image path
image2_path = 'C:/brain_tumor/version2/notebook/try_img/image_10.png'  # Update with your image path

img1 = cv2.imread(image1_path)
img2 = cv2.imread(image2_path)

# Perform fusion
fused_image = image_fusion(img1, img2)

# Display the original and fused images
cv2.imshow('Image 1', img1)
cv2.imshow('Image 2', img2)
cv2.imshow('Fused Image', fused_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [62]:
import cv2
import numpy as np

def Sobel(img):
    # Apply Sobel edge detection
    return cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=3)

def Gaussian_Function(x, mu=0, sigma=1):
    # Calculate Gaussian function value
    return np.exp(-((x - mu)**2) / (2 * sigma**2))

def image_fusion(images, metrics):
    gaussian_matrices = []
    weight_matrices = []

    # Edge detection and Gaussian matrix filling
    for img in images:
        edge_img = Sobel(img)
        gaussian_matrix = np.zeros_like(edge_img, dtype=float)
        for y in range(edge_img.shape[0]):
            for x in range(edge_img.shape[1]):
                g = Gaussian_Function(edge_img[y, x])
                gaussian_matrix[y, x] = g
        gaussian_matrices.append(gaussian_matrix)

    # Weight matrix calculation
    metric_arr = metrics
    for flag in range(2):
        weight_matrix = np.zeros_like(gaussian_matrices[flag], dtype=float)
        for y in range(weight_matrix.shape[0]):
            for x in range(weight_matrix.shape[1]):
                g1 = gaussian_matrices[0][y, x]
                g2 = gaussian_matrices[1][y, x]
                if g1 != 0 and g2 != 0:
                    w = g1 / (g1 + g2)
                else:
                    t = metric_arr[flag] / (metric_arr[0] + metric_arr[1])
                    w = t
                weight_matrix[y, x] = w
        weight_matrices.append(weight_matrix)

    # Image fusion
    Agg_img = np.zeros_like(images[0], dtype=float)
    for y in range(Agg_img.shape[0]):
        for x in range(Agg_img.shape[1]):
            if images[0][y, x] == images[1][y, x]:
                Agg_img[y, x] = images[0][y, x] * weight_matrices[0][y, x] + images[1][y, x] * weight_matrices[1][y, x]

    return Agg_img

# Example usage
image1 = cv2.imread("C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png", cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread("C:/brain_tumor/version2/notebook/try_img/image_10.png", cv2.IMREAD_GRAYSCALE)
images = [image1, image2]  # Example images
metrics = [0.5542362183803938, 0.6147776098315835]  # Example metrics of images
result_image = image_fusion(images, metrics)

# Display the result image
cv2.imshow("Result Image", result_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [45]:
import numpy as np
from scipy.ndimage import sobel

img1 = cv2.imread("C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("C:/brain_tumor/version2/notebook/try_img/image_10.png", cv2.IMREAD_GRAYSCALE)

def Sobel(img):
    return sobel(img)

def Gaussian_Function(pixel):
    # Define your Gaussian function here
    # For simplicity, let's assume a standard Gaussian function
    return np.exp(-(pixel**2) / (2 * (sigma**2)))

# Assuming you have two images stored as img1 and img2

# Initialize variables
gaussian_matrix = [[], []]
weight_matrix = [[], []]
metric_arr = [0.5542362183803938, 0.6147776098315835]

# Compute Gaussian matrices
for flag, img in enumerate([img1, img2]):
    edge_img = Sobel(img)
    for pixel in np.nditer(edge_img):
        g = Gaussian_Function(pixel)
        gaussian_matrix[flag].append(g)

# Compute weight matrices
for flag in range(2):
    for index, g in enumerate(gaussian_matrix[flag]):
        g1 = gaussian_matrix[0][index]
        g2 = gaussian_matrix[1][index]
        if g1 != 0 and g2 != 0:
            w = g / (g1 + g2)
        else:
            t = metric_arr[flag] / (metric_arr[0] + metric_arr[1])
            w = t
        weight_matrix[flag].append(w)

# Aggregate images
Agg_img = np.zeros_like(img1)
for pixel_1, pixel_2, w_1, w_2 in zip(np.nditer(img1), np.nditer(img2), weight_matrix[0], weight_matrix[1]):
    if pixel_1 == pixel_2:
        Agg_img[np.unravel_index(pixel_1, Agg_img.shape)] = pixel_1 * w_1 + pixel_2 * w_2

# Agg_img will contain the aggregated image
# cv2.imshow("Result Image", Agg_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

print("pixel1", pixel_1)
print("pixel2", pixel_2)

C:\Users\admin\AppData\Local\Temp\ipykernel_15940\366795163.py:13: RuntimeWarning: overflow encountered in scalar negative
  return np.exp(-(pixel**2) / (2 * (sigma**2)))


pixel1 12
pixel2 2


In [39]:
import cv2
from skimage.metrics import structural_similarity as ssim
# Read images
img1 = cv2.imread('C:/brain_tumor/version2/notebook/vanilla_gan/glio_gen_imgs/images_6_1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('C:/brain_tumor/version2/notebook/try_img/image_10.png', cv2.IMREAD_GRAYSCALE)

# Compute SSIM
ssim_value_1_to_2 = ssim(img1, img2)
ssim_value_2_to_1 = ssim(img2, img1)

print("SSIM value from image1 to image2:", ssim_value_1_to_2)
print("SSIM value from image2 to image1:", ssim_value_2_to_1)


SSIM value from image1 to image2: 0.4975106948506672
SSIM value from image2 to image1: 0.4975106948506672
